***
## Learning without Forgetting - LwF

Experiment reproducing **Learning without Forgetting** method:  
hybrid of Distillation Networks and fine-tuning, which refers to the re-training with a low learning rate an already trained model M with new and more specific dataset, D<sub>new</sub>, with respect to the dataset, D<sub>old</sub>, with which the given model M was originally trained.

LwF, as opposed to other continual learning techniques, only uses the new data, so it assumes that past data used to pre-train the network is unavailable.  
It is a *transfer learning technique*.

`References:`
- Learning without Forgetting: https://arxiv.org/abs/1606.09282
- Three scenarios for continual learning: https://arxiv.org/abs/1904.07734
***

### Arguments to customize training
import argparse
parser = argparse.ArgumentParser()

parser.add_argument(**'--lwf_alpha'**, nargs='+', type=float,
                    default=[0, 0.5, 1.333, 2.25, 3.2],
                    help='Penalty hyperparameter for LwF. It can be either
                    a list with multiple elements (one alpha per 
                    experience) or a list of one element (same alpha 
                    for all experiences).')

parser.add_argument(**'--softmax_temperature'**, type=float, default=1,
                    help='Temperature for softmax used in distillation')

parser.add_argument(**'--lr'**, type=float, default=1e-3, help='Learning rate.')

parser.add_argument(**'--hs'**, type=int, default=256, help='MLP hidden size.')

parser.add_argument(**'--epochs'**, type=int, default=10,
                    help='Number of training epochs.')

parser.add_argument(**'--minibatch_size'**, type=int, default=128,
                    help='Minibatch size.')

parser.add_argument(**'--cuda'**, type=int, default=0,
                    help='Specify GPU id to use. Use CPU if -1.')

args = parser.parse_args()
***

In [2]:
# Testing framework and test runner
import unittest

# Avalanche library
import avalanche
from avalanche.evaluation import metrics as metrics

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD

# NN model (Multilayer perceptron)
from models import MLP 

from utils import arguments, get_average_metric, get_target_result

## LwF technique
class LwF(unittest.TestCase): #TestCase class

    #### Split MNIST benchmark
    def test_smnist(self, override_args=None):
        
        # -> BENCHMARK
        # stream composed of 5 experiences from SplitMNIST dataset
        benchmark = avalanche.benchmarks.SplitMNIST(n_experiences=5, return_task_id=False)
               
        # --- Strategy instantiation --- # 
        # 1. Model
        # 2. Optimizer
        # 3. Loss function
        
        # -> ADDITIONAL ARGUMENTS allow to customize training
        args = arguments({ 'cuda': 0,              # GPU
                           'lwf_alpha': 1,         # Penalty hyperparameter for LwF
                           'lwf_temperature': 1,   # Temperature for softmax used in distillation
                           'epochs': 10,           # Training epochs
                           'layers': 1,             
                           'hidden_size': 256,     # MLP hidden size
                           'learning_rate': 0.001, # Learning rate
                           'train_mb_size': 128}, override_args) # Minibatch size

        # Set up and run CUDA operations,
        # if CUDA is available, utilize GPUs for computation.
        device = torch.device(f"cuda:{args.cuda}"
                              if torch.cuda.is_available() and args.cuda >= 0 
                              else "cpu")
        
        model = MLP(hidden_size=args.hidden_size, hidden_layers=args.layers)
        optimizer = SGD(model.parameters(), lr=args.learning_rate)
        criterion = CrossEntropyLoss()
        
        # ---------------- LOG and EVALUATION ---------------- #
        # Avalanche logging module, displays a progress bar during training and evaluation
        interactive_logger = avalanche.logging.InteractiveLogger()
        
        # -> EVALUATION
        # Metrics of main interest to be tracked
        evaluation_plugin = avalanche.training.plugins.EvaluationPlugin(
            metrics.accuracy_metrics(epoch=True, experience=True, stream=True),
            loggers=[interactive_logger], benchmark=benchmark)
        # ---------------------------------------------------- #

        # -> CONTINUAL LEARNING STRATEGY: LwF
        cl_strategy = avalanche.training.LwF(model,
                                             optimizer,
                                             criterion,
                                             # additional arguments
                                             alpha=args.lwf_alpha, 
                                             temperature=args.lwf_temperature,
                                             train_mb_size=args.train_mb_size, 
                                             train_epochs=args.epochs,
                                             device=device,
                                             # evaluation
                                             #evaluator=evaluation_plugin,
                                            )
        
        # --- Training loop --- #
        print('Starting experiment...')
        for experience in benchmark.train_stream:
            print('Start of experience: ', experience.current_experience)
            print('Current Classes:     ', experience.classes_in_this_experience)
            # train
            cl_strategy.train(experience)
            print('Training completed')
            # test (returns a dictionary which contains all the metric values)
            res = cl_strategy.eval(benchmark.test_stream)

        # access the metric values
        #avg_stream_acc = get_average_metric(res)
        #print(f"LwF-SMNIST Average Stream Accuracy: {avg_stream_acc:.2f}")

        # compare the metric with the target values
        #target_acc = float(get_target_result('lwf', 'smnist'))
        #if args.check:
        #    self.assertAlmostEqual(target_acc, avg_stream_acc, delta=0.02) #delta is too small??

### Run and Evaluate the experiment
- Create an instance of the strategy object
- Execute the strategy on a benchmark

In [3]:
# Create the strategy
s = LwF()

# Run the experiment with custom parameters
s.test_smnist()

# ??? What shall I see ???
# Training on experience 0 (num: 0, 1)
# Evaluate on experience 0,1,2,3,4
# Training on experience 1 (num: 2,3)
# Evaluate on experience 0,1,2,3,4
# .....


Starting experiment...
Start of experience:  0
Current Classes:      [3, 6]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|███████████████████████████████████████████| 95/95 [00:02<00:00, 32.54it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3791
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7976
100%|███████████████████████████████████████████| 95/95 [00:02<00:00, 33.02it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4354
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9820
100%|███████████████████████████████████████████| 95/95 [00:02<00:00, 33.12it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2295
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9828
100%|███████████████████████████████████████████| 95/95 [00:02<00:00, 32.64it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1600
	Top1_Acc_Epoch/train_phase/train_stream/Task00

-- Starting training on experience 2 (Task 0) from train stream --
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 30.92it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3438
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3503
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 31.55it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7149
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9630
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 31.66it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.6105
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9670
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 31.42it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.5687
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9668
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 31.47it/s]
Epoch 4 ended.
	Loss

100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 31.89it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.2292
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1189
100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 31.85it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3216
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7169
100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 31.74it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1275
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8107
100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 31.41it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0525
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8297
100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 30.88it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0103
	Top1_Acc_Epoch/tr